In [1]:
import os
import sys
import datetime
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm.notebook import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, MultiStepLR, CyclicLR, LambdaLR

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import accuracy_score, roc_auc_score



## Build Dataset

In [2]:
train_data=pd.read_csv("/kaggle/input/playground-series-s4e3/train.csv")
test_data=pd.read_csv("/kaggle/input/playground-series-s4e3/test.csv")

In [3]:
train_data.head().T

,0,1,2,3,4
id,0.0000,1.0000,2.000000e+00,3.000000e+00,4.0000
X_Minimum,584.0000,808.0000,3.900000e+01,7.810000e+02,1540.0000
X_Maximum,590.0000,816.0000,1.920000e+02,7.890000e+02,1560.0000
Y_Minimum,909972.0000,728350.0000,2.212076e+06,3.353146e+06,618457.0000
Y_Maximum,909977.0000,728372.0000,2.212144e+06,3.353173e+06,618502.0000
Pixels_Areas,16.0000,433.0000,1.138800e+04,2.100000e+02,521.0000
X_Perimeter,8.0000,20.0000,7.050000e+02,1.600000e+01,72.0000
Y_Perimeter,5.0000,54.0000,4.200000e+02,2.900000e+01,67.0000
Sum_of_Luminosity,2274.0000,44478.0000,1.311391e+06,3.202000e+03,48231.0000
Minimum_of_Luminosity,113.0000,70.0000,2.900000e+01,1.140000e+02,82.0000


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19219 entries, 0 to 19218
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     19219 non-null  int64  
 1   X_Minimum              19219 non-null  int64  
 2   X_Maximum              19219 non-null  int64  
 3   Y_Minimum              19219 non-null  int64  
 4   Y_Maximum              19219 non-null  int64  
 5   Pixels_Areas           19219 non-null  int64  
 6   X_Perimeter            19219 non-null  int64  
 7   Y_Perimeter            19219 non-null  int64  
 8   Sum_of_Luminosity      19219 non-null  int64  
 9   Minimum_of_Luminosity  19219 non-null  int64  
 10  Maximum_of_Luminosity  19219 non-null  int64  
 11  Length_of_Conveyer     19219 non-null  int64  
 12  TypeOfSteel_A300       19219 non-null  int64  
 13  TypeOfSteel_A400       19219 non-null  int64  
 14  Steel_Plate_Thickness  19219 non-null  int64  
 15  Ed

In [5]:
# selecting features and labels
X=train_data.iloc[:,1:-7]
y=train_data.iloc[:,-7:]

# Train test split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=13)

# Standardize features
std_scaler=StandardScaler()
X_train_scaled=std_scaler.fit_transform(X_train)
X_val_scaled=std_scaler.transform(X_val)

# Building tensors (FP16)
X_train_tensor=torch.as_tensor(X_train_scaled,dtype=torch.float32)
X_val_tensor=torch.as_tensor(X_val_scaled,dtype=torch.float32)
y_train_tensor=torch.as_tensor(y_train.values,dtype=torch.float32)
y_val_tensor=torch.as_tensor(y_val.values,dtype=torch.float32)



In [6]:
class SPD_dataset(Dataset):
    def __init__(self,x_tensor,y_tensor):
        self.x=x_tensor
        self.y=y_tensor
        
    def __getitem__(self,idx):
        return (self.x[idx],self.y[idx])
    
    def __len__(self):
        return len(self.x)

In [172]:
# Build tensor datasets
train_dataset=SPD_dataset(X_train_tensor,y_train_tensor)
val_dataset=SPD_dataset(X_val_tensor,y_val_tensor)

# Build dataloaders
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=128,shuffle=False)

## Build Model Class

In [8]:
X_train.shape,y_train.shape

((15375, 27), (15375, 7))

In [93]:
class AnuwazNet(nn.Module):
    def __init__(self):
        super(AnuwazNet,self).__init__()
        
        self.feature_extractor=nn.Sequential(
            nn.Linear(27,64,dtype=torch.float32),
            nn.BatchNorm1d(64,dtype=torch.float32),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64,32,dtype=torch.float32),
            nn.BatchNorm1d(32,dtype=torch.float32),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.classifier=nn.Sequential(nn.Linear(32,7),
                                      nn.Sigmoid())
        
        
    def forward(self,x):
        x=self.feature_extractor(x)
        x=self.classifier(x)
        return x
        
        
    

## Model Training

In [94]:
model=AnuwazNet()
criterion=nn.BCELoss(reduction='mean')
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [95]:
print(model)

AnuwazNet(
  (feature_extractor): Sequential(
    (0): Linear(in_features=27, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=32, out_features=7, bias=True)
    (1): Sigmoid()
  )
)


In [96]:
summary(model,input_size=(128,27))

Layer (type:depth-idx)                   Output Shape              Param #
AnuwazNet                                [128, 7]                  --
├─Sequential: 1-1                        [128, 32]                 --
│    └─Linear: 2-1                       [128, 64]                 1,792
│    └─BatchNorm1d: 2-2                  [128, 64]                 128
│    └─ReLU: 2-3                         [128, 64]                 --
│    └─Dropout: 2-4                      [128, 64]                 --
│    └─Linear: 2-5                       [128, 32]                 2,080
│    └─BatchNorm1d: 2-6                  [128, 32]                 64
│    └─ReLU: 2-7                         [128, 32]                 --
│    └─Dropout: 2-8                      [128, 32]                 --
├─Sequential: 1-2                        [128, 7]                  --
│    └─Linear: 2-9                       [128, 7]                  231
│    └─Sigmoid: 2-10                     [128, 7]                  --
Total p

In [ ]:
train_losses=[]
val_losses=[]

# put model to training model
model.train()

for i in tqdm(range(100)):
    for features,labels in train_loader:
        optimizer.zero_grad()

        # forward pass
        yhat=model(features)
        # loss
        loss=criterion(yhat,labels)
        # backpropagation
        loss.backward()
        # Update parameters
        optimizer.step()
    
    # put model to evaluation mode
    model.eval()
    val_aucs=[]
    
    with torch.no_grad():
        for features, labels_val in val_loader:
            # prediction
            yhat_val=model(features)
            # validation loss
            val_loss=criterion(yhat,labels)
            
            try:
                val_aucs.append(roc_auc_score(labels_val,yhat_val, multi_class='ovo'))
            except ValueError:
                pass
            
    val_auc=np.mean(val_aucs)
    print(f"Epoch: {i+1} -------------------- Validation AUC-ROC: {val_auc}")
    
    




  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 -------------------- Validation AUC-ROC: 0.8639090437226248
Epoch: 2 -------------------- Validation AUC-ROC: 0.8672576450479274
Epoch: 3 -------------------- Validation AUC-ROC: 0.8677655937146025
Epoch: 4 -------------------- Validation AUC-ROC: 0.8680341230970436
Epoch: 5 -------------------- Validation AUC-ROC: 0.8676560803801743
Epoch: 6 -------------------- Validation AUC-ROC: 0.867779546959729
Epoch: 7 -------------------- Validation AUC-ROC: 0.8666214185907626
Epoch: 8 -------------------- Validation AUC-ROC: 0.8676867191130128
Epoch: 9 -------------------- Validation AUC-ROC: 0.8659777677397289
Epoch: 10 -------------------- Validation AUC-ROC: 0.8672875778983407
Epoch: 11 -------------------- Validation AUC-ROC: 0.8653899762970221
Epoch: 12 -------------------- Validation AUC-ROC: 0.8651215132847837
Epoch: 13 -------------------- Validation AUC-ROC: 0.8653992064162235
Epoch: 14 -------------------- Validation AUC-ROC: 0.8650854074232357
Epoch: 15 --------------------

In [173]:
with torch.no_grad():
    for features, labels_val in val_loader:
        print(labels_val.shape)

torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([4, 7])


In [174]:
with torch.no_grad():
    for features, labels_val in val_loader:
        print(model(features).shape)

torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([128, 7])
torch.Size([4, 7])


In [98]:
next(iter(train_loader))[1][0,:].detach().numpy()

array([0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [99]:
model.eval()
with torch.no_grad():
    print(model(next(iter(train_loader))[0])[0,:].detach().numpy())

[6.0865790e-02 2.6119121e-03 2.7320525e-01 9.5278352e-05 7.6140444e-05
 2.0242545e-01 3.0335578e-01]


In [166]:
auc_=[]
for i in range(7):
    #print(roc_auc_score(next(iter(val_loader))[1][:,i].detach().numpy(),model(next(iter(val_loader))[0])[:,i].detach().numpy()))
    auc_.append(roc_auc_score(next(iter(val_loader))[1][:,i].detach().numpy(),model(next(iter(val_loader))[0])[:,i].detach().numpy()))
print(np.mean(auc_))

0.8881026376621733


In [161]:
print(roc_auc_score(next(iter(val_loader))[1].detach().numpy(),model(next(iter(val_loader))[0]).detach().numpy()))

0.8881026376621733


In [152]:
std_scaler.transform(test_data.iloc[:,1:])

array([[ 0.57632991,  0.56037032,  1.02996671, ...,  1.17028833,
        -0.72473625,  1.04247234],
       [ 1.03194572,  1.03673356, -0.74591062, ...,  1.64960447,
        -0.05300309,  1.02199456],
       [ 1.22209942,  1.23888771, -0.90347223, ..., -1.28477054,
         0.79331419,  0.2420319 ],
       ...,
       [-1.25742959, -1.08688575, -0.13724874, ..., -0.33906504,
         0.48903902,  1.29181948],
       [ 1.16750083,  1.17483887, -0.5987584 , ...,  1.52608207,
        -1.03067413,  0.64255326],
       [-1.26119501, -1.13292086,  0.00458825, ...,  0.02862953,
         0.28036944,  1.29181948]])

In [102]:
model.eval()
with torch.no_grad():
    pred=model(torch.as_tensor(std_scaler.transform(test_data.iloc[:,1:]),dtype=torch.float32))

In [108]:
pred_np=pred.detach().cpu().numpy()
pred_np.shape

(12814, 7)

In [116]:
np.sum(pred_np,axis=1).max(),np.sum(pred_np,axis=1).min(),np.sum(pred_np,axis=1).mean()

(1.9999403, 0.0061494466, 0.9526588)

In [104]:
submission=pd.read_csv("/kaggle/input/playground-series-s4e3/sample_submission.csv")
submission.columns.tolist()[1:]

['Pastry',
 'Z_Scratch',
 'K_Scatch',
 'Stains',
 'Dirtiness',
 'Bumps',
 'Other_Faults']

In [117]:
for i,label in enumerate(submission.columns.tolist()[1:]):
    submission[label]=pred[:,i]

In [118]:
submission.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.663519,0.000079,0.000015,0.000003,0.039203,0.139452,0.264554
1,19220,0.304662,0.002762,0.002049,0.000008,0.044306,0.089520,0.398456
2,19221,0.000140,0.032324,0.126352,0.001912,0.011147,0.218353,0.563823
3,19222,0.217191,0.000054,0.000009,0.000008,0.014130,0.521746,0.280649
4,19223,0.000808,0.000359,0.000035,0.000288,0.003660,0.857188,0.199919


In [119]:
submission.to_csv('submission.csv',index=False)

In [120]:
submission.tail()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
12809,32028,0.182325,2.996436e-01,0.001202,1.060180e-06,5.438252e-02,2.290133e-01,0.328001
12810,32029,0.298282,4.692870e-03,0.008280,4.636799e-03,1.294745e-01,2.020261e-01,0.371940
12811,32030,0.000004,2.713402e-07,0.960688,5.314574e-07,1.686841e-08,1.004243e-07,0.034837
12812,32031,0.476097,1.047329e-03,0.006456,1.011466e-06,2.189654e-02,2.098419e-01,0.226553
12813,32032,0.000020,3.510934e-04,0.960130,1.862395e-07,2.121194e-07,6.285071e-05,0.039123


In [ ]:
roc_auc_score()